[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ianjure/stock-market-trend-forecast/blob/main/Stock_Market_Trend_Forecast_Notebook.ipynb)

**Author:** Ian Jure Macalisang

**Email:** ianjuremacalisang2@gmail.com

**Link:** https://github.com/ianjure/stock-market-trend-prediction

**Website:** https://predictstocktrend.streamlit.app/

# **I. Data Collection**

We will access the Yahoo Finance API using the yfinance library to fetch historical stock price data. All ticker symbols are found [here](https://stockanalysis.com/stocks/).

In [1]:
import yfinance as yf
import pandas as pd
pd.set_option('display.max_columns', 500)

import warnings
warnings.filterwarnings('ignore')

In [99]:
ticker = 'goog' # @param {type:"string"}
timeframe = 'Month' # @param ["Week", "Month"]

In [100]:
# FETCH DATA
stock = yf.Ticker(ticker)
stock_name = stock.info['longName']
stock_ticker = stock.info['symbol']
stock = stock.history(period="max")

# **II. Data Preprocessing**

In [101]:
# CHECK IF THE STOCK IS PUBLICLY TRADED FOR MORE THAN 2 YEARS
if stock.shape[0] < 520:
  print("The stock is not publicly traded for more than 2 years.")
else:
  print("The stock is publicly traded for more than 2 years.")

The stock is publicly traded for more than 2 years.


In [102]:
def format_week(stock):

    # CLEANING
    del stock["Dividends"]
    del stock["Stock Splits"]
    stock.index = stock.index.strftime('%Y-%m-%d')
    stock.index = pd.to_datetime(stock.index)
    stock_date = pd.date_range(start="1990-01-01", end=stock.index[-1].tz_localize(None), freq="W-FRI")
    stock = stock.reindex(stock_date, method="ffill")

    # FEATURE ENGINEERING
    stock["Next Week"] = stock["Close"].shift(-1)
    stock["Target"] = (stock["Next Week"] > stock["Close"]).astype(int)

    horizons = [4,13,26,52,208]

    for horizon in horizons:
      rolling_averages = stock.rolling(horizon).mean()

      ratio_column = f"Close_Ratio_{horizon}"
      stock[ratio_column] = stock["Close"] / rolling_averages["Close"]

      trend_column = f"Trend_{horizon}"
      stock[trend_column] = stock.shift(1).rolling(horizon).sum()["Target"]

    stock = stock.dropna(subset=['Close_Ratio_4', 'Trend_4', 'Close_Ratio_13', 'Trend_13', 'Close_Ratio_26', 'Trend_26', 'Close_Ratio_52', 'Trend_52', 'Close_Ratio_208', 'Trend_208'])

    # SCALING
    from sklearn.preprocessing import StandardScaler, MinMaxScaler

    features_to_scale = ["Open","High","Low","Close","Volume"]
    standard_scaler = StandardScaler()
    minmax_scaler = MinMaxScaler()

    standard_scaled = standard_scaler.fit_transform(stock[features_to_scale])
    standard_scaled = pd.DataFrame(standard_scaled, index=stock.index, columns=['Standard Open', 'Standard High', 'Standard Low', 'Standard Close', 'Standard Volume'])

    minmax_scaled = minmax_scaler.fit_transform(stock[features_to_scale])
    minmax_scaled = pd.DataFrame(minmax_scaled, index=stock.index, columns=['MinMax Open', 'MinMax High', 'MinMax Low', 'MinMax Close', 'MinMax Volume'])

    stock = pd.concat([stock, standard_scaled], axis=1)
    stock = pd.concat([stock, minmax_scaled], axis=1)

    # ADDING FEATURES
    from fredapi import Fred

    API_KEY = 'YOUR FRED API KEY HERE'
    fred = Fred(api_key=API_KEY)

    crude_oil = fred.get_series('DCOILWTICO').rename('Price')
    crude_oil = crude_oil.reindex_like(stock)
    stock['Crude Oil'] = crude_oil
    stock['Crude Oil'] = crude_oil.ffill()
    stock = stock.dropna(subset=['Crude Oil'])

    effective_rate = fred.get_series('DFF').rename('Rate')
    effective_rate = effective_rate.reindex_like(stock)
    stock['Effective Rate'] = effective_rate
    stock['Effective Rate'] = effective_rate.ffill()

    interest_rate = fred.get_series('T10Y3M').rename('Rate')
    interest_rate = interest_rate.reindex_like(stock)
    stock['Interest Rate'] = interest_rate
    stock['Interest Rate'] = interest_rate.ffill()

    return stock

def format_month(stock):

    # CLEANING
    del stock["Dividends"]
    del stock["Stock Splits"]
    stock.index = stock.index.strftime('%Y-%m-%d')
    stock.index = pd.to_datetime(stock.index)
    stock_date = pd.date_range(start="1990-01-01", end=stock.index[-1].tz_localize(None), freq="M")
    stock = stock.reindex(stock_date, method="ffill")
    stock["Next Month"] = stock["Close"].shift(-1)
    stock["Target"] = (stock["Next Month"] > stock["Close"]).astype(int)

    # FEATURE ENGINEERING
    horizons = [2,6,12,30,60]

    for horizon in horizons:
      rolling_averages = stock.rolling(horizon).mean()

      ratio_column = f"Close_Ratio_{horizon}"
      stock[ratio_column] = stock["Close"] / rolling_averages["Close"]

      trend_column = f"Trend_{horizon}"
      stock[trend_column] = stock.shift(1).rolling(horizon).sum()["Target"]

    stock = stock.dropna(subset=['Close_Ratio_2', 'Trend_2', 'Close_Ratio_6', 'Trend_6', 'Close_Ratio_12', 'Trend_12', 'Close_Ratio_30', 'Trend_30', 'Close_Ratio_60', 'Trend_60'])

    # SCALING
    from sklearn.preprocessing import StandardScaler, MinMaxScaler

    features_to_scale = ["Open","High","Low","Close","Volume"]
    standard_scaler = StandardScaler()
    minmax_scaler = MinMaxScaler()

    standard_scaled = standard_scaler.fit_transform(stock[features_to_scale])
    standard_scaled = pd.DataFrame(standard_scaled, index=stock.index, columns=['Standard Open', 'Standard High', 'Standard Low', 'Standard Close', 'Standard Volume'])

    minmax_scaled = minmax_scaler.fit_transform(stock[features_to_scale])
    minmax_scaled = pd.DataFrame(minmax_scaled, index=stock.index, columns=['MinMax Open', 'MinMax High', 'MinMax Low', 'MinMax Close', 'MinMax Volume'])

    stock = pd.concat([stock, standard_scaled], axis=1)
    stock = pd.concat([stock, minmax_scaled], axis=1)

    # ADDING FEATURES
    from fredapi import Fred

    API_KEY = 'YOUR FRED API KEY HERE'
    fred = Fred(api_key=API_KEY)

    crude_oil = fred.get_series('DCOILWTICO').rename('Price')
    crude_oil = crude_oil.reindex_like(stock)
    stock['Crude Oil'] = crude_oil
    stock['Crude Oil'] = crude_oil.ffill()
    stock = stock.dropna(subset=['Crude Oil'])

    effective_rate = fred.get_series('DFF').rename('Rate')
    effective_rate = effective_rate.reindex_like(stock)
    stock['Effective Rate'] = effective_rate
    stock['Effective Rate'] = effective_rate.ffill()

    interest_rate = fred.get_series('T10Y3M').rename('Rate')
    interest_rate = interest_rate.reindex_like(stock)
    stock['Interest Rate'] = interest_rate
    stock['Interest Rate'] = interest_rate.ffill()

    return stock

In [103]:
stock = format_week(stock) if timeframe == 'Week' else format_month(stock)
stock

,Open,High,Low,Close,Volume,Next Month,Target,Close_Ratio_2,Trend_2,Close_Ratio_6,Trend_6,Close_Ratio_12,Trend_12,Close_Ratio_30,Trend_30,Close_Ratio_60,Trend_60,Standard Open,Standard High,Standard Low,Standard Close,Standard Volume,MinMax Open,MinMax High,MinMax Low,MinMax Close,MinMax Volume,Crude Oil,Effective Rate,Interest Rate
2009-07-31,11.194753,11.262422,11.006921,11.022346,114844859.0,11.485579,1,1.024820,2.0,1.124545,5.0,1.174628,7.0,0.932998,16.0,1.109213,35.0,-1.033051,-1.033628,-1.036751,-1.038548,1.412677,0.000000,0.000000,0.000000,0.000000,0.401435,69.26,0.18,3.34
2009-08-31,11.438809,11.490306,11.394277,11.485579,78609548.0,12.335921,1,1.020581,2.0,1.113544,6.0,1.224432,8.0,0.971376,17.0,1.138757,36.0,-1.027535,-1.028526,-1.027887,-1.028043,0.523093,0.001398,0.001300,0.002248,0.002687,0.274537,69.97,0.15,3.25
2009-09-30,12.439167,12.442650,12.121719,12.335921,126139035.0,13.337772,1,1.035696,2.0,1.128916,6.0,1.287947,9.0,1.040542,17.0,1.204924,36.0,-1.004925,-1.007206,-1.011241,-1.008760,1.689952,0.007130,0.006732,0.006469,0.007619,0.440987,70.46,0.07,3.17
2009-10-31,13.683084,13.687312,13.291000,13.337772,139260032.0,14.504067,1,1.039022,2.0,1.158965,6.0,1.341230,10.0,1.119976,17.0,1.284803,36.0,-0.976810,-0.979342,-0.984485,-0.986040,2.012075,0.014258,0.013831,0.013255,0.013431,0.486938,70.46,0.11,3.17
2009-11-30,14.445107,14.520737,14.357534,14.504067,69262644.0,15.424069,1,1.041890,2.0,1.189278,6.0,1.375349,11.0,1.210737,17.0,1.375125,37.0,-0.959587,-0.960684,-0.960080,-0.959591,0.293624,0.018624,0.018585,0.019444,0.020196,0.241803,77.19,0.13,3.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-31,151.827230,152.496467,151.157994,152.086929,21105600.0,164.452866,1,1.042734,1.0,1.095409,4.0,1.149957,8.0,1.243335,15.0,1.476203,36.0,2.145498,2.128201,2.170299,2.160450,-0.888641,0.805800,0.805590,0.813328,0.818251,0.073154,79.22,5.33,-1.20
2024-04-30,167.189757,169.676917,164.313025,164.452866,29420800.0,173.762268,1,1.039066,2.0,1.131119,5.0,1.200817,8.0,1.338465,15.0,1.569546,36.0,2.492719,2.512823,2.471323,2.440879,-0.684501,0.893825,0.903586,0.889669,0.889980,0.102275,83.49,5.33,-0.77
2024-05-31,173.202892,174.221737,170.775661,173.762268,28085200.0,183.419998,1,1.027525,2.0,1.142757,5.0,1.230943,8.0,1.402260,16.0,1.627677,37.0,2.628627,2.614568,2.619206,2.651993,-0.717291,0.928279,0.929509,0.927173,0.943980,0.097597,77.97,5.33,-0.95
2024-06-30,185.720001,186.580002,183.324997,183.419998,23032400.0,173.149994,0,1.027039,2.0,1.152399,5.0,1.253062,9.0,1.464867,16.0,1.684112,38.0,2.911537,2.891235,2.906370,2.871007,-0.841338,1.000000,1.000000,1.000000,1.000000,0.079902,77.97,5.33,-0.95


In [104]:
# CHECK FOR NULL VALUES
print(stock.isnull().sum())

# CHECK THE NUMBER OF ROWS AND COLUMNS
print(stock.shape)

Open               0
High               0
Low                0
Close              0
Volume             0
Next Month         1
Target             0
Close_Ratio_2      0
Trend_2            0
Close_Ratio_6      0
Trend_6            0
Close_Ratio_12     0
Trend_12           0
Close_Ratio_30     0
Trend_30           0
Close_Ratio_60     0
Trend_60           0
Standard Open      0
Standard High      0
Standard Low       0
Standard Close     0
Standard Volume    0
MinMax Open        0
MinMax High        0
MinMax Low         0
MinMax Close       0
MinMax Volume      0
Crude Oil          0
Effective Rate     0
Interest Rate      0
dtype: int64
(181, 30)


# **III. Model Training**

In [105]:
"""
BACKTESTING FUNCTION
"""

def predict(train, test, predictors, model):
    model.fit(train[predictors], train["Target"])
    preds = model.predict_proba(test[predictors])[:,1]
    conf = preds.copy()
    conf = pd.Series(conf, index=test.index, name="Confidence")
    preds[preds >= .6] = 1
    preds[preds < .6] = 0
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    combined = pd.concat([combined, conf], axis=1)
    return combined

def backtest(data, model, predictors, timeframe):
    all_predictions = []
    if timeframe == 'Week':
        for i in range(520, data.shape[0], 52):
            train = data.iloc[0:i].copy()
            test = data.iloc[i:(i+52)].copy()
            predictions = predict(train, test, predictors, model)
            all_predictions.append(predictions)
    else:
        for i in range(120, data.shape[0], 12):
            train = data.iloc[0:i].copy()
            test = data.iloc[i:(i+12)].copy()
            predictions = predict(train, test, predictors, model)
            all_predictions.append(predictions)
    combined = pd.concat(all_predictions)
    return combined

In [106]:
# IMPORTING THE MODELS AND TOOLS
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score

In [107]:
# INITIALIZE PREDICTORS
if timeframe == 'Week':
    predictors = {
        'Original': ['Open', 'High', 'Low', 'Close', 'Volume']	,
        'Standard Scaled': ['Standard Open', 'Standard High', 'Standard Low', 'Standard Close', 'Standard Volume'],
        'MinMax Scaled': ['MinMax Open', 'MinMax High', 'MinMax Low', 'MinMax Close', 'MinMax Volume'],
        'Ratio Trend': ['Close_Ratio_4', 'Trend_4', 'Close_Ratio_13', 'Trend_13', 'Close_Ratio_26', 'Trend_26', 'Close_Ratio_52', 'Trend_52', 'Close_Ratio_208', 'Trend_208'],
        'Economic Indicators': ['Crude Oil', 'Effective Rate', 'Interest Rate'],
        'Ratio Trend Economic Indicators': ['Close_Ratio_4', 'Trend_4', 'Close_Ratio_13', 'Trend_13', 'Close_Ratio_26', 'Trend_26', 'Close_Ratio_52', 'Trend_52', 'Close_Ratio_208', 'Trend_208', 'Crude Oil', 'Effective Rate', 'Interest Rate'],
        'Ratio Trend Interest Rate': ['Close_Ratio_4', 'Trend_4', 'Close_Ratio_13', 'Trend_13', 'Close_Ratio_26', 'Trend_26', 'Close_Ratio_52', 'Trend_52', 'Close_Ratio_208', 'Trend_208', 'Interest Rate'],
        'Ratio Trend Crude Oil Rate': ['Close_Ratio_4', 'Trend_4', 'Close_Ratio_13', 'Trend_13', 'Close_Ratio_26', 'Trend_26', 'Close_Ratio_52', 'Trend_52', 'Close_Ratio_208', 'Trend_208', 'Crude Oil'],
        'Ratio Trend Effective Rate': ['Close_Ratio_4', 'Trend_4', 'Close_Ratio_13', 'Trend_13', 'Close_Ratio_26', 'Trend_26', 'Close_Ratio_52', 'Trend_52', 'Close_Ratio_208', 'Trend_208', 'Effective Rate'],
        'Ratio Trend Effective Interest': ['Close_Ratio_4', 'Trend_4', 'Close_Ratio_13', 'Trend_13', 'Close_Ratio_26', 'Trend_26', 'Close_Ratio_52', 'Trend_52', 'Close_Ratio_208', 'Trend_208', 'Effective Rate', 'Interest Rate'],
        'Ratio Trend Crude Oil Interest': ['Close_Ratio_4', 'Trend_4', 'Close_Ratio_13', 'Trend_13', 'Close_Ratio_26', 'Trend_26', 'Close_Ratio_52', 'Trend_52', 'Close_Ratio_208', 'Trend_208', 'Crude Oil', 'Interest Rate'],
        'Ratio Trend Crude Oil Effective': ['Close_Ratio_4', 'Trend_4', 'Close_Ratio_13', 'Trend_13', 'Close_Ratio_26', 'Trend_26', 'Close_Ratio_52', 'Trend_52', 'Close_Ratio_208', 'Trend_208', 'Crude Oil', 'Effective Rate']
    }
else:
    predictors = {
        'Original': ['Open', 'High', 'Low', 'Close', 'Volume']	,
        'Standard Scaled': ['Standard Open', 'Standard High', 'Standard Low', 'Standard Close', 'Standard Volume'],
        'MinMax Scaled': ['MinMax Open', 'MinMax High', 'MinMax Low', 'MinMax Close', 'MinMax Volume'],
        'Ratio Trend': ['Close_Ratio_2', 'Trend_2', 'Close_Ratio_6', 'Trend_6', 'Close_Ratio_12', 'Trend_12', 'Close_Ratio_30', 'Trend_30', 'Close_Ratio_60', 'Trend_60'],
        'Economic Indicators': ['Crude Oil', 'Effective Rate', 'Interest Rate'],
        'Ratio Trend Economic Indicators': ['Close_Ratio_2', 'Trend_2', 'Close_Ratio_6', 'Trend_6', 'Close_Ratio_12', 'Trend_12', 'Close_Ratio_30', 'Trend_30', 'Close_Ratio_60', 'Trend_60', 'Crude Oil', 'Effective Rate', 'Interest Rate'],
        'Ratio Trend Interest Rate': ['Close_Ratio_2', 'Trend_2', 'Close_Ratio_6', 'Trend_6', 'Close_Ratio_12', 'Trend_12', 'Close_Ratio_30', 'Trend_30', 'Close_Ratio_60', 'Trend_60', 'Interest Rate'],
        'Ratio Trend Crude Oil Rate': ['Close_Ratio_2', 'Trend_2', 'Close_Ratio_6', 'Trend_6', 'Close_Ratio_12', 'Trend_12', 'Close_Ratio_30', 'Trend_30', 'Close_Ratio_60', 'Trend_60', 'Crude Oil'],
        'Ratio Trend Effective Rate': ['Close_Ratio_2', 'Trend_2', 'Close_Ratio_6', 'Trend_6', 'Close_Ratio_12', 'Trend_12', 'Close_Ratio_30', 'Trend_30', 'Close_Ratio_60', 'Trend_60', 'Effective Rate'],
        'Ratio Trend Effective Interest': ['Close_Ratio_2', 'Trend_2', 'Close_Ratio_6', 'Trend_6', 'Close_Ratio_12', 'Trend_12', 'Close_Ratio_30', 'Trend_30', 'Close_Ratio_60', 'Trend_60', 'Effective Rate', 'Interest Rate'],
        'Ratio Trend Crude Oil Interest': ['Close_Ratio_2', 'Trend_2', 'Close_Ratio_6', 'Trend_6', 'Close_Ratio_12', 'Trend_12', 'Close_Ratio_30', 'Trend_30', 'Close_Ratio_60', 'Trend_60', 'Crude Oil', 'Interest Rate'],
        'Ratio Trend Crude Oil Effective': ['Close_Ratio_2', 'Trend_2', 'Close_Ratio_6', 'Trend_6', 'Close_Ratio_12', 'Trend_12', 'Close_Ratio_30', 'Trend_30', 'Close_Ratio_60', 'Trend_60', 'Crude Oil', 'Effective Rate']
    }

# INITIALIZE CLASSIFIERS
classifiers = {
    'Logistic Regression': LogisticRegression(random_state=1),
    'KNN': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(random_state=1),
    'Gradient Boosting': GradientBoostingClassifier(random_state=1)
}

In [108]:
"""
TRAINING ALL MODELS WITH ALL PREDICTORS
"""

# SPLITTING THE DATASET
train = stock.iloc[:-100] # get all the data except the last 100 rows as the training set
test = stock.iloc[-100:] # get the last 100 rows as the testing set

top_results = {'Classifier': [],
               'Predictors': [],
               'Accuracy': [],
               'Precision': [],
               'Total Errors': []
               }

# ITERATE OVER PREDICTORS
for predictor_name, prd in predictors.items():

    # DICTIONARY TO STORE INITIAL RESULTS
    results = {'Classifier': [],
               'Predictors': [],
               'Accuracy': [],
               'Precision': [],
               'Total Errors': []
               }

    # ITERATE OVER CLASSIFIERS
    for clf_name, clf in classifiers.items():

        # FIT THE DATA TO THE MODEL AND BACKTEST
        preds = backtest(stock, clf, prd, timeframe)

        # CALCULATE PRECISION AND ERRORS
        accuracy = accuracy_score(preds["Target"], preds["Predictions"])
        precision = precision_score(preds["Target"], preds["Predictions"])
        errors = (preds["Target"] != preds["Predictions"]).sum()

        # STORE RESULTS
        results['Classifier'].append(clf_name)
        results['Predictors'].append(predictor_name)
        results['Accuracy'].append(accuracy)
        results['Precision'].append(precision)
        results['Total Errors'].append(errors)

    # CREATE DATAFRAME FROM RESULTS
    results_df = pd.DataFrame(results).sort_values(by='Accuracy', ascending=False).reset_index(drop=True)

    # APPEND TO TOP RESULT
    top_results['Classifier'].append(results['Classifier'][0])
    top_results['Predictors'].append(results['Predictors'][0])
    top_results['Accuracy'].append(results['Accuracy'][0])
    top_results['Precision'].append(results['Precision'][0])
    top_results['Total Errors'].append(results['Total Errors'][0])

# DISPLAY THE TOP RESULTS
top_results_df = pd.DataFrame(top_results).sort_values(by='Accuracy', ascending=False).reset_index(drop=True)
top_results_df[['Classifier', 'Predictors', 'Accuracy', 'Precision', 'Total Errors']]

,Classifier,Predictors,Accuracy,Precision,Total Errors
0,Logistic Regression,Ratio Trend Economic Indicators,0.606557,0.606557,24
1,Logistic Regression,Ratio Trend Crude Oil Interest,0.606557,0.606557,24
2,Logistic Regression,Standard Scaled,0.573770,0.617021,26
3,Logistic Regression,MinMax Scaled,0.557377,0.604167,27
4,Logistic Regression,Economic Indicators,0.557377,0.619048,27
5,Logistic Regression,Ratio Trend Interest Rate,0.540984,0.578947,28
6,Logistic Regression,Ratio Trend Effective Interest,0.540984,0.578947,28
7,Logistic Regression,Ratio Trend Crude Oil Rate,0.524590,0.605263,29
8,Logistic Regression,Ratio Trend Crude Oil Effective,0.475410,0.551020,32
9,Logistic Regression,Ratio Trend,0.442623,0.542857,34


In [109]:
# TRAINING THE FINAL MODEL WITH THE WHOLE DATASET
final_model = classifiers[top_results_df['Classifier'][0]]

# **IV. Model Testing**

In [110]:
result = backtest(stock, final_model, predictors[top_results_df['Predictors'][0]], timeframe)

In [111]:
result

,Target,Predictions,Confidence
2019-07-31,0,1.0,0.696285
2019-08-31,1,1.0,0.719212
2019-09-30,1,1.0,0.733366
2019-10-31,1,1.0,0.708219
2019-11-30,1,1.0,0.732207
...,...,...,...
2024-03-31,1,1.0,0.778238
2024-04-30,1,1.0,0.722064
2024-05-31,1,1.0,0.719551
2024-06-30,0,1.0,0.744721


In [114]:
if timeframe == 'Week':
    if result['Predictions'][-1] > 0:
        print(f"{stock_name} ({stock_ticker}) stock price will go up next week.")
        print(f"Confidence: {round(result['Confidence'][-1] * 100)}%")
    else:
        print(f"{stock_name} ({stock_ticker}) stock price will go down next week.")
        print(f"Confidence: {round((1 - result['Confidence'][-1]) * 100)}%")
else:
    if result['Predictions'][-1] > 0:
        print(f"{stock_name} ({stock_ticker}) stock price will go up next month.")
        print(f"Confidence: {round(result['Confidence'][-1] * 100)}%")
    else:
        print(f"{stock_name} ({stock_ticker}) stock price will go down next month.")
        print(f"Confidence: {round((1 - result['Confidence'][-1]) * 100)}%")

Alphabet Inc. (GOOG) stock price will go up next month.
Confidence: 71%
